In [2]:
import rope.base.project
from rope.refactor.extract import ExtractMethod


In [3]:
myproject = rope.base.project.Project('src')

myresource = myproject.get_resource('test.py')
print(myresource.read().find("b"), myresource.read().find("print"))

extractor = ExtractMethod(myproject, myresource, 28, 64)
change_set = extractor.get_changes("helper")
for change in change_set.changes:
    print(change.get_description())

28 64
--- a/test.py
+++ b/test.py
@@ -1,5 +1,9 @@
 def a_func():
     a = 1
+    b, c = helper(a)
+    print(b, c)
+
+def helper(a):
     b = 2 * a
     c = a * 2 + b * 3
-    print(b, c)
+    return b, c



In [6]:
def extract_method(
    snippet,
    file_path,
    method_name,
    project_name="../../sweepai"
):
    project = rope.base.project.Project(project_name)

    resource = project.get_resource(file_path)
    contents = resource.read()
    start, end = contents.find(snippet), contents.find(snippet) + len(snippet)

    extractor = ExtractMethod(project, resource, start, end)
    change_set = extractor.get_changes(method_name, similar=True, global_=True)
    for change in change_set.changes:
        change.do()

    result = resource.read()

    for change in change_set.changes:
        change.undo()
    return result

snippet = """if not DEBUG:
        logger.info("Hydrating cache of sandbox.")
        try:
            requests.post(
                SANDBOX_URL,
                json={
                    "repo_url": f"https://github.com/{repo_full_name}",
                    "token": user_token,
                },
                timeout=2,
            )
        except Timeout:
            logger.info("Sandbox hydration timed out.")
        except SystemExit:
            raise SystemExit
        except Exception as e:
            logger.warning(
                f"Error hydrating cache of sandbox (tracking ID: `{tracking_id}`): {e}"
            )
        logger.info("Done sending, letting it run in the background.")"""
print(extract_method(snippet, "handlers/on_ticket.py", "hydrate_sandbox"))

"""
on_ticket is the main function that is called when a new issue is created.
It is only called by the webhook handler in sweepai/api.py.
"""

import math
import re
import traceback
from time import time

import openai
import requests
import yaml
import yamllint.config as yamllint_config
from github import BadCredentialsException
from logtail import LogtailContext, LogtailHandler
from loguru import logger
from requests.exceptions import Timeout
from tabulate import tabulate
from tqdm import tqdm
from yamllint import linter

from sweepai.config.client import (
    DEFAULT_RULES,
    RESET_FILE,
    RESTART_SWEEP_BUTTON,
    REVERT_CHANGED_FILES_TITLE,
    RULES_LABEL,
    RULES_TITLE,
    SWEEP_BAD_FEEDBACK,
    SWEEP_GOOD_FEEDBACK,
    SweepConfig,
    get_documentation_dict,
    get_rules,
)
from sweepai.config.server import (
    DEBUG,
    DISCORD_FEEDBACK_WEBHOOK_URL,
    ENV,
    GITHUB_BOT_USERNAME,
    GITHUB_LABEL_NAME,
    IS_SELF_HOSTED,
    LOGTAIL_SOURCE_KEY,
    MONGODB_U